### Regresión logística

En el presente script se trabajará con el dataset de _Pima Indians Diabetes_ extraído de [Kaggle](https://www.kaggle.com/datasets/uciml/Pima-indians-diabetes-database). Este conjunto de datos es originalmente del Instituto Nacional de Diabetes y Enfermedades Digestivas y Renales. El objetivo del conjunto de datos es predecir diagnósticamente si un paciente tiene o no diabetes, en función de ciertas mediciones diagnósticas incluidas en el conjunto de datos. En particular, todos los pacientes del dataset son mujeres de al menos 21 años de edad de ascendencia indígena Pima.

Las variables presentes son:
* Cantidad de embarazos - Pregnancies
* Concentración de glucosa - Glucose
* Presión arterial - BloodPressure
* Grosor de la piel - SkinThickness
* Nivel de insulina - Insulin
* Índice de masa corporal - BMI
* Medida del historial de diabetes en la familia - DiabetesPedigreeFunction
* Edad en años - Age
* Variable objetivo - Outcome, donde 0 implica no diabetes y 1 diabetes

In [1]:
# Comenzamos importando las librerías a utilizar
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
